In [1]:
using TensorNetworks, DoubleFloats, LinearAlgebra, TensorOperations

┌ Info: Precompiling TensorNetworks [bf05fd6d-b814-440f-8e78-cc5435f5625f]
└ @ Base loading.jl:1342
  ** incremental compilation may be fatally broken for this module **



In [2]:
T = ComplexF64
Dmaxhigh = 100;
Dmaxlow = 20;
tol = 1e-15;
d=2
truncationhigh = TruncationArgs(Dmaxhigh,tol,true)
truncationlow = TruncationArgs(Dmaxlow,tol,true)

TruncationArgs(20, 1.0e-15, true)

In [3]:
w, u = TensorNetworks.constr_wu_dim2(T)
@tensor U[:] := u[-1, -3, -5, -7] * u[-2, -4, -6, -8]
d = size(u, 1)
U = Gate(reshape(U, d^2, d^2, d^2, d^2))
@tensor W[:] := w[-1, -3, -5] * w[-2, -4, -6]
W = reshape(W, d^2, d^2, d^2);

In [4]:
function coarse_grain(mps, U, W)
    if length(mps) == 1
        mps = TensorNetworks.double(mps)
    end
    mps = TensorNetworks.apply_layers_nonunitary(mps, [[U]])
    A = data(mps.Γ[1])
    B = data(GenericSite(mps[2],:right))
    # lA = mps.Λ[1]
    lB = mps.Λ[2]
    # B = B*lA
    @tensor C[:] := B[-1, ul, c] * W[-2, ul, ur] * A[c, ur, -3]
    return UMPS([GenericSite(C,false)], [lB], mps)
end
coarse_grain_uw(mps) = coarse_grain(mps,U,W)

coarse_grain_uw (generic function with 1 method)

In [5]:
A0 = A = B = reshape(
    permutedims(
        reshape(kron(Matrix{T}(I, d, d), Matrix{T}(I, d, d)), d, d, d, d),
        [1, 3, 2, 4]),
    d,d^2,d)
lA0 = lA = lB = ones(T,d) / sqrt(d)
mpshigh = UMPS([A0], [lA0], truncation=truncationhigh)

MPS: UMPS{ComplexF64}
Sites: OrthogonalLinkSite{ComplexF64}
Length: 1
Truncation: TruncationArgs(100, 1.0e-15, true)

In [6]:
target=coarse_grain_uw(coarse_grain_uw(mpshigh))

MPS: UMPS{ComplexF64}
Sites: OrthogonalLinkSite{ComplexF64}
Length: 1
Truncation: TruncationArgs(100, 1.0e-15, true)

In [7]:
mpslow = deepcopy(target)
mpslow.truncation = truncationlow
guess = TensorNetworks.canonicalize(mpslow)

MPS: UMPS{ComplexF64}
Sites: OrthogonalLinkSite{ComplexF64}
Length: 1
Truncation: TruncationArgs(20, 1.0e-15, true)